# Tokenize pre-mRNA sequences

In [ ]:
import sys
sys.path.append('/work/home/rnasys/zhouhanwen/github/LAMAR_baselines/RNAErnie')
from tokenization_rnaernie import RNAErnieTokenizer
from datasets import load_dataset, Dataset
import multiprocessing as mp
import pandas as pd
import random
import json
import os

In [ ]:
os.chdir('/work/home/rnasys/zhouhanwen/github/LAMAR_baselines/')

In [ ]:
# tokenizer
tokenizer_path = 'tokenizer/RNAErnie/'
model_max_length = 1026
tokenizer = RNAErnieTokenizer.from_pretrained('tokenizer/RNAErnie/', model_max_length=model_max_length)

In [ ]:
def group_texts(examples):
    tokenized_inputs = tokenizer(examples["seq"], truncation=True, max_length=tokenizer.model_max_length)
    tokenized_inputs["labels"] = examples["label"]
    return tokenized_inputs

In [ ]:
# paths of training set and testing set
train_set_path = 'SpliceSitePred/data/gencode_train.json'
# path of saving file
data_path = 'SpliceSitePred/data/RNAErnie/ss_single_nucleotide'

In [ ]:
dataset = load_dataset("json", data_files=train_set_path)

In [ ]:
data = dataset.map(group_texts, batched=True, num_proc=mp.cpu_count())

In [ ]:
data = data.remove_columns(["seq", "label"])

In [ ]:
data = data["train"].train_test_split(test_size=0.001, shuffle=True, seed=42)

In [ ]:
data.save_to_disk(data_path, num_proc=1)

In [ ]:
data